In [1]:
import os
import sys
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from natsort import natsorted
import string

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from natsort import natsorted
import string
import os

def read_file(filename):
    with open(filename, 'r', encoding="ascii", errors="surrogateescape") as f:
        stuff = f.read()

    f.close()

    return stuff

def preprocessing(final_string):
    # Tokenize.
    tokenizer = TweetTokenizer()
    token_list = tokenizer.tokenize(final_string)

    # Remove punctuations.
    table = str.maketrans('', '', '\t')
    token_list = [word.translate(table) for word in token_list]
    punctuations = (string.punctuation).replace("'", "")
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in token_list]
    token_list = [str for str in stripped_words if str]

    # Change to lowercase.
    token_list = [word.lower() for word in token_list]
    return token_list


# In this example, we create the positional index for only 1 folder.
folder_names = ["data"]

#stopwords
stop_words = set(stopwords.words('english'))

# Initialize the stemmer.
stemmer = PorterStemmer()

# Initialize the file no.
fileno = 0

# Initialize the vocabulary.
pos_index = {}

# Initialize the file mapping (fileno -> file name).
file_map = {}

for folder_name in folder_names:

    # Open files.
    file_names = natsorted(os.listdir(folder_name))

    # For every file.
    for file_name in file_names:
        
#         print(file_name)
        # Read file contents.
        if os.path.isdir(file_name):
            continue
        stuff = read_file(folder_name + '/' + file_name)

        # This is the list of words in order of the text.
        # We need to preserve the order because we require positions.
        # 'preprocessing' function does some basic punctuation removal,
        # stopword removal etc.
        final_token_list = preprocessing(stuff)
#         print(final_token_list[0:20])
        
        token_list_removed_stop_word = [] #Removal stop_word
        for term in final_token_list:
            if term not in stop_words: #check words if it is stop word
                token_list_removed_stop_word.append(term)
                
#         print(token_list_removed_stop_word[0:20])
#         break
        # For position and term in the tokens.
        for pos, term in enumerate(token_list_removed_stop_word):
            # First stem the term.
            term = stemmer.stem(term)

            # If term already exists in the positional index dictionary.
            if term in pos_index:

                # Increment total freq by 1.
                pos_index[term][0] = pos_index[term][0] + 1

                # Check if the term has existed in that DocID before.
                if fileno in pos_index[term][1]:
                    pos_index[term][1][fileno].append(pos)

                else:
                    pos_index[term][1][fileno] = [pos]

            # If term does not exist in the positional index dictionary
            # (first encounter).
            else:

                # Initialize the list.
                pos_index[term] = []
                # The total frequency is 1.
                pos_index[term].append(1)
                # The postings list is initially empty.
                pos_index[term].append({})
                # Add doc ID to postings list.
                pos_index[term][1][fileno] = [pos]

                # Map the file no. to the file name.
        file_map[fileno] = file_name

        # Increment the file no. counter for document ID mapping
        fileno += 1

print(file_map)

{0: 'Artificial intelligence.txt', 1: 'Computer Science.txt', 2: 'Computer Vision.txt', 3: 'Convolutional_neural_network.txt', 4: 'Deep Learning.txt', 5: 'Information_retrieval.txt', 6: 'Natural_language_processing.txt', 7: 'Operation System.txt', 8: 'clustering.txt', 9: 'machine learning.txt'}


In [2]:
# Sample positional index to test the code.
sample_pos_idx = pos_index["system"]
print("Positional Index")

# print(sample_pos_idx)

file_list = sample_pos_idx[1]
print("Filename, [Positions]")
# print(file_list)
print(file_list.get(0))
# for fileno, positions in file_list.items():
#     print(fileno, positions)

print()

Positional Index
Filename, [Positions]
[108, 654, 739, 795, 816, 895, 997, 1004, 1146, 1157, 1169, 1174, 1185, 1189, 1191, 1449, 1459, 1565, 1643, 1849, 1933, 2113, 2309, 2637, 2700, 2757, 2851, 3355, 3368, 3371, 3403, 3563, 3565, 4162, 4251, 4381, 4387, 4536, 4936, 5164, 5241, 5553]



In [7]:
import copy

# Start of Boolean Queries
query = "deep neural network and learn not science"

# Tokenizing the the query
normalized_query = preprocessing(query)
print("Boolean Query after normalization:", normalized_query)

# 'connecting_words' means operations (and, or, not)
connecting_terms = []
cnt = 1

# 'different_words' means search words
different_terms = []

# Splitting the search terms and boolean operations (and, not, or)
temp_list = []
for term in normalized_query:
    # First stem the query terms
    
    term = stemmer.stem(term)

    if term != "and" and term != "or" and term != "not":
        temp_list.append(term) # when term is word
    else: # when term is connective
        connecting_terms.append(term)
        
        different_terms.append(copy.deepcopy(temp_list))
        temp_list.clear()

different_terms.append(copy.deepcopy(temp_list))
temp_list.clear()

print("Separated terms:", different_terms)
print("Boolean operators:", connecting_terms)
print()

# get all unique words from inverted index dictionary
unique_words_all = set(pos_index.keys())
# print(unique_words_all)
# number of documents
total_files = fileno + 1  # len(files_with_index)

# index of documents, if search word occurs 1, else 0.
zeroes_and_ones = []

# index of documents for all search words
zeroes_and_ones_of_all_terms = []

# Find ids for each search word
for term in (different_terms):
    
    if len(term) > 1 : #if term is connected word
        zeroes_and_ones = [0] * total_files
        docID_list = []
        if term[0] in unique_words_all: #tropical --> fish
#             print('term : ', term[0])
            sample_pos_idx = pos_index[term[0]]
            for docID, position in sample_pos_idx[1].items(): # docID : 1, position : [1, 5, 10 ....]
                #다음 단어의 docid에서 pos + x가 존재하는지 체크
#                 print("docID : ", docID)
                for p in position:
#                     print()
#                     print("{} --> ".format(p))
                    isCorrect = False
                    for x in range(1, len(term)):
                        search_pos_index = pos_index[term[x]][1].get(docID)
                        if search_pos_index is None or not (p + x in search_pos_index):
                            break;     
#                         print('next term : ', term[x])
#                         print("{} --> ".format(p + x))
                        if x == len(term) - 1:
                            isCorrect = True
                            
                    if isCorrect == True:
#                         print('append DocID :', docID)
                        docID_list.append(docID)
                        break
                        
            for docID in docID_list:
                zeroes_and_ones[docID] = 1
            zeroes_and_ones_of_all_terms.append(zeroes_and_ones)
                    
    else:           
        term = term[0]
#         print(term)
        if term in unique_words_all:
            # First set 0 to all documents
            zeroes_and_ones = [0] * total_files
            sample_pos_idx = pos_index[term]

            # Set 1 for document index, if search word occurs
            for docId in sample_pos_idx[1].keys():
                zeroes_and_ones[docId] = 1

            zeroes_and_ones_of_all_terms.append(zeroes_and_ones)

        else:
            print(term, " not found")
            sys.exit()

# print(zeroes_and_ones_of_all_terms)
print("Vectors for query terms", zeroes_and_ones_of_all_terms)
print()

# Checking intersections
for term in connecting_terms:

    # Results of search word after or before operation
    word_list1 = zeroes_and_ones_of_all_terms[0]
    word_list2 = zeroes_and_ones_of_all_terms[1]

    if term == "and": #0 & 0 = 0 1 & 1 = 1 0 & 1 = 0 1 & 0 = 0
        bitwise_op = [w1 & w2 for (w1, w2) in zip(word_list1, word_list2)]
        zeroes_and_ones_of_all_terms.remove(word_list1)
        zeroes_and_ones_of_all_terms.remove(word_list2)
        zeroes_and_ones_of_all_terms.insert(0, bitwise_op)

    elif term == "or":
        bitwise_op = [w1 | w2 for (w1, w2) in zip(word_list1, word_list2)]
        zeroes_and_ones_of_all_terms.remove(word_list1)
        zeroes_and_ones_of_all_terms.remove(word_list2)
        zeroes_and_ones_of_all_terms.insert(0, bitwise_op)

    elif term == "not":
        bitwise_op = [not w1 for w1 in word_list2]
        bitwise_op = [int(b == True) for b in bitwise_op]
        zeroes_and_ones_of_all_terms.remove(word_list2)
        zeroes_and_ones_of_all_terms.remove(word_list1)
        bitwise_op = [w1 & w2 for (w1, w2) in zip(word_list1, bitwise_op)]
        zeroes_and_ones_of_all_terms.insert(0, bitwise_op)

intersections = zeroes_and_ones_of_all_terms[0]
print("Intersections", intersections)
print()

files = []
cnt = 0
for index in intersections:
    if index == 1:
        files.append(file_names[cnt])
    cnt = cnt + 1

print("Search words were found in ", files)

Boolean Query after normalization: ['deep', 'neural', 'network', 'and', 'learn', 'not', 'science']
Separated terms: [['deep', 'neural', 'network'], ['learn'], ['scienc']]
Boolean operators: ['and', 'not']

Vectors for query terms [[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0], [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0], [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0]]

Intersections [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

Search words were found in  ['Convolutional_neural_network.txt']
